# A)

In [89]:
import pandas as pd

In [90]:
trainlabels = pd.read_csv('TrainLabels.csv')
print(trainlabels)

        id                                              tweet label
0        1  The CDC currently reports 99031 deaths. In gen...  real
1        2  States reported 1121 deaths a small rise from ...  real
2        3  Politically Correct Woman (Almost) Uses Pandem...  fake
3        4  #IndiaFightsCorona: We have 1524 #COVID testin...  real
4        5  Populous states can generate large case counts...  real
...    ...                                                ...   ...
6415  6416  A tiger tested positive for COVID-19 please st...  fake
6416  6417  ???Autopsies prove that COVID-19 is??� a blood...  fake
6417  6418  _A post claims a COVID-19 vaccine has already ...  fake
6418  6419  Aamir Khan Donate 250 Cr. In PM Relief Cares Fund  fake
6419  6420  It has been 93 days since the last case of COV...  real

[6420 rows x 3 columns]


# B)

In [91]:
import nltk
import re
import html
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [92]:
fake_labels = trainlabels[trainlabels['label'] == 'real']
fake_labels = trainlabels[trainlabels['label'] == 'fake']                                                                               

In [93]:
fake_labels['tokenized_tweets'] = fake_labels['tweet'].apply(lambda x: word_tokenize(x))
fake_labels['tokenized_tweets'] = fake_labels['tweet'].apply(lambda x: word_tokenize(x))


C:\Users\Danny Tapp\AppData\Local\Temp\ipykernel_15736\809847916.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_labels['tokenized_tweets'] = fake_labels['tweet'].apply(lambda x: word_tokenize(x))
C:\Users\Danny Tapp\AppData\Local\Temp\ipykernel_15736\809847916.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_labels['tokenized_tweets'] = fake_labels['tweet'].apply(lambda x: word_tokenize(x))


In [94]:
real_unique_words = set([word for tweet in real_labels['tokenized_tweets'] for word in tweet]) 
fake_unique_words = set([word for tweet in fake_labels['tokenized_tweets'] for word in tweet]) 
real_unique_count = len(real_unique_words)
fake_uniqe_count = len(fake_unique_words)
print("Real Unique Words Count:", real_unique_count)
print("Fake Unique Words Count:", fake_uniqe_count)


Real Unique Words Count: 14859
Fake Unique Words Count: 13258


In [95]:
real_avg_words = real_labels['tokenized_tweets'].apply(len).mean()
fake_avg_words = fake_labels['tokenized_tweets'].apply(len).mean()
print("Average Words per Real Post:",real_avg_words)
print("Average Words per Fake Post:",fake_avg_words)

Average Words per Real Post: 38.49404761904762
Average Words per Fake Post: 26.362418300653594


In [96]:
real_avg_chars = real_labels['tweet'].apply(len).mean()
fake_avg_chars = fake_labels['tweet'].apply(len).mean()
print("Average Characters per Real Post:",real_avg_chars)
print("Average Characters per Fake Post:",fake_avg_chars)

Average Characters per Real Post: 215.0613095238095
Average Characters per Fake Post: 144.8875816993464


#### Fake posts appear to have lower words per post and characters per post in comparison to their real counterparts. 
#### The total unique word count is also lower for fake tweets. 

# C)

In [97]:
stop_words = set(stopwords.words('english'))

In [98]:
def remove_urls(text):
    text = re.sub(r'http[s]?://\S+|www\.S+','',text)
    text = re.sub(r'@\w+','',text)
    return text

In [99]:
def xml_lemmatize(text):
    text = html.unescape(text)
    tokens = word_tokenize(text)
    cleaned_tokens = [
        lemmatizer.lemmatize(word.lower()) for word in tokens
        if word.lower() not in stop_words and word.isalpha()
    ]

    return cleaned_tokens

In [100]:
real_labels.loc[:,'cleaned_tokenized_tweets'] = real_labels['tweet'] \
    .apply(remove_urls) \
    .apply(xml_lemmatize)

fake_labels.loc[:,'cleaned_tokenized_tweets'] = fake_labels['tweet'] \
    .apply(remove_urls) \
    .apply(xml_lemmatize)

C:\Users\Danny Tapp\AppData\Local\Temp\ipykernel_15736\2118398705.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_labels.loc[:,'cleaned_tokenized_tweets'] = fake_labels['tweet'] \


# E)

In [101]:
import textstat

In [102]:
real_labels['flesch_reading_ease'] = real_labels.loc[:,'cleaned_tokenized_tweets'].apply(lambda x: textstat.flesch_reading_ease(' '.join(x)))
real_labels['flesch_kincaid_grade'] = real_labels.loc[:,'cleaned_tokenized_tweets'].apply(lambda x: textstat.flesch_kincaid_grade(' '.join(x)))
real_labels['gunning_fog'] = real_labels.loc[:,'cleaned_tokenized_tweets'].apply(lambda x: textstat.gunning_fog(' '.join(x)))
real_labels['smog_index'] = real_labels.loc[:,'cleaned_tokenized_tweets'].apply(lambda x: textstat.smog_index(' '.join(x)))
real_labels['syllable_count'] = real_labels.loc[:,'cleaned_tokenized_tweets'].apply(lambda x: textstat.syllable_count(' '.join(x)))
real_labels['sentence_count'] = real_labels.loc[:,'cleaned_tokenized_tweets'].apply(lambda x: textstat.sentence_count(' '.join(x)))

In [103]:
fake_labels['flesch_reading_ease'] = fake_labels.loc[:,'cleaned_tokenized_tweets'].apply(lambda x: textstat.flesch_reading_ease(' '.join(x)))
fake_labels['flesch_kincaid_grade'] = fake_labels.loc[:,'cleaned_tokenized_tweets'].apply(lambda x: textstat.flesch_kincaid_grade(' '.join(x)))
fake_labels['gunning_fog'] = fake_labels.loc[:,'cleaned_tokenized_tweets'].apply(lambda x: textstat.gunning_fog(' '.join(x)))
fake_labels['smog_index'] = fake_labels.loc[:,'cleaned_tokenized_tweets'].apply(lambda x: textstat.smog_index(' '.join(x)))
fake_labels['syllable_count'] = fake_labels.loc[:,'cleaned_tokenized_tweets'].apply(lambda x: textstat.syllable_count(' '.join(x)))
fake_labels['sentence_count'] = fake_labels.loc[:,'cleaned_tokenized_tweets'].apply(lambda x: textstat.sentence_count(' '.join(x)))

C:\Users\Danny Tapp\AppData\Local\Temp\ipykernel_15736\4024435890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_labels['flesch_reading_ease'] = fake_labels.loc[:,'cleaned_tokenized_tweets'].apply(lambda x: textstat.flesch_reading_ease(' '.join(x)))
C:\Users\Danny Tapp\AppData\Local\Temp\ipykernel_15736\4024435890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_labels['flesch_kincaid_grade'] = fake_labels.loc[:,'cleaned_tokenized_tweets'].apply(lambda x: textstat.flesch_kincaid_grade(' '.

### F)

In [104]:
all_labels = pd.concat([real_labels,fake_labels],axis = 0)

In [105]:
avg_read_scores = all_labels.groupby('label')[['flesch_reading_ease','flesch_kincaid_grade','gunning_fog','smog_index','syllable_count','sentence_count']].mean()
print(avg_read_scores)

       flesch_reading_ease  flesch_kincaid_grade  gunning_fog  smog_index  \
label                                                                       
fake             22.157605             13.191307    15.481516         0.0   
real             20.596833             14.191488    15.373884         0.0   

       syllable_count  sentence_count  
label                                  
fake        24.454902             1.0  
real        30.869048             1.0  


#### The flesch kincaid grade shows that real posts are averaging a higher grade level than fake posts. The flesch reading ease has real posts slightly more confusing to read than fake posts, but that may be because fake posts are trying to be as accessible as possible. Real posts average more syllables than fake posts, meaning they may use more complex words in their tweets. 